In [33]:
import pandas as pd
import netCDF4
import datetime
import numpy as np
import os

## Write Met File

In [ ]:
tccon_data_fullpath = '/uufs/chpc.utah.edu/common/home/u0890904/LAIR_1/Data/TCCON/ci20230502_20230701.private.nc'
f = netCDF4.Dataset(tccon_data_fullpath)

In [45]:
#Unit,UTCDate,UTCTime,WSPD,WDIR,SigTheta,Gust,Tout,RH,Pout,Sflux,Precip,LeafWet,Battery,Bit
# format the time 
time = f.variables['time']
datetimes = netCDF4.num2date(time[:], time.units)
datetimes = [date.strftime('%Y-%m-%d %H:%M:%S') for date in datetimes]

# list the variables you want to keep
cols = list( ['tout','pout','hout','wspd','wdir'] )

# extract those variables from the netcdf and append together
list_nc = []
for c in cols:
    list_nc.append(list(f.variables[c][:]))
    
# convert into pandas dataframe
df = pd.DataFrame(list_nc)
df = df.T # transpose the dataframe
df.columns = cols # set column names

# add the time as the dataframe index 
df['dt'] = pd.to_datetime(datetimes, utc = True)
df = df.rename(columns = {'tout':'Tout','pout':'Pout','wspd':'WSPD','wdir':'WDIR','hout':'RH'})
df['UTCDate'] = df.apply(lambda row: datetime.datetime.strftime(row['dt'],'%y/%m/%d'),axis = 1)
df['UTCTime'] = df.apply(lambda row: datetime.datetime.strftime(row['dt'],'%H:%M:%S'),axis = 1)
# subset the time period of interest 
sub_df = df.loc[(df['UTCDate']=='23/05/16')]
sub_df = sub_df[['UTCDate','UTCTime','Tout','Pout','RH','WSPD','WDIR']]

In [46]:
met_save_path = '/uufs/chpc.utah.edu/common/home/lin-group15/agm/em27/ua/inst_data/met/tccon/'
dates = ['2023-05-16','2023-05-17','2023-05-18','2023-05-19']
sub_dfs = {}
for date in dates:
    sub_df = df.loc[(df['dt']>=f'{date} 00:00:00')&(df['dt']<=f'{date} 23:59:59')]
    sub_df = sub_df[['UTCDate','UTCTime','Tout','Pout','RH','WSPD','WDIR']]
    filename = f"{date.replace('-','')}_tccon.txt"
    fullsavepath = os.path.join(met_save_path,filename)
    sub_df.to_csv(fullsavepath,index=False)